# Evaluate Model with Politics 101

In [42]:
import numpy as np
from tqdm import tqdm
import cv2
#Input the nyp arrays
truth = np.load("labels.npy")
images = np.load("images.npy")

labels = truth.copy()
all_images = images.shape[0]
threshold = 0.4
print("The number of samples/images/instances for our valuation dataset are {}".format(all_images))
iou_threshold = [0.1, 0.4, 0.3] #these are the thresold numbers passed in the MTCNN
predictions = []
recall_missed = []
precision = []
IOU = []
tp = [] #face when it was face
fp = [] #face when it wasn't face
tn = [] #not face when it was not face
fn = [] #not face when it was face


'''Test one image '''
i =0
test = images[i]
#resize from (320,320,3) to (32,32,3)
test = cv2.resize(test,(320,320))
rectangles = detectFace(test,iou_threshold)
if len(rectangles)!=0:
    #print(rectangles)
    #print(labels[i][0], labels[i][1],labels[i][2],labels[i][3])
    cv2.imwrite("test.jpg",test)

    cv2.rectangle(test,(labels[i][0],labels[i][1]),(labels[i][0]+labels[i][2],labels[i][1]+labels[i][3]),(255,0,0),2)
    cv2.rectangle(test,(int(rectangles[0][0]),int(rectangles[0][1])),(int(rectangles[0][2]),int(rectangles[0][3])),(0,0,255),2)
    cv2.imshow("test",test)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''End of test'''


for i in tqdm(range(0,all_images)):
    img = images[i]
    #resize from (320,320,3) to (32,32,3)
    #img = cv2.resize(img,(32,32))
    #get ground truth coordinates
    x1 = labels[i][0]
    y1 = labels[i][1]
    x2 = labels[i][0]+labels[i][2]
    y2 = labels[i][1]+labels[i][3]
    co  = labels[i][4]
    rectangles = detectFace(img, iou_threshold)
    #print("Rectangles output shape is",np.array(rectangles).shape)
    num_of_faces  = np.array(rectangles).shape[0]
    for j in range(0,num_of_faces):
        pred_x1 = rectangles[j][0]
        pred_y1 = rectangles[j][1]
        pred_x2 = rectangles[j][2]
        pred_y2 = rectangles[j][3]
        pred_c = rectangles[j][4]
        #ignore for now the landmark positioning
        iou = IoU(pred_x1,pred_y1,pred_x2,pred_y2, x1,y1,x2,y2)
        #print(iou)
        if iou>=threshold:
            predictions.append(1)
            precision.append(1)
            tp.append(1)
            IOU.append(iou)
            
        else:
            predictions.append(1)
            fp.append(1)
            IOU.append(iou)
        save_all(img,pred_x1,pred_y1,pred_x2,pred_y2, x1,y1,x2,y2,co)
        #I = I[np.where(o<=iou_threshold)[0]]
    
    if num_of_faces==0:
        fn.append(1)
        IOU.append(0)
        predictions.append(1)
'''Account for the case where there is no face and we predicted face'''
tn = [0]
predictions.append(0)
'''End of case'''

#print the accuracy of the model
recall = sum(tp)/(sum(tp)+sum(fn)) #true positive and false negative
accuracy = (sum(tp)+sum(tn))/sum(predictions)
precision = sum(tp)/(sum(fp)+sum(tp))
f1_score = 2*recall*precision/(recall+precision)
print("The achieved accuracy for threshold {} was {}".format(0.5, accuracy))
print("The achieved recall for threshold {} was {}".format(0.5, recall))
print("The achieved precision was {}".format(precision))
print("F1  score is {}".format(f1_score))
print("fp {} , tn {}, tp {} , fn {}".format(sum(fp), sum(tn), sum(tp), sum(fn)))
print("Average IoU achieved was {}%".format(np.mean(np.array(IOU))))

The number of samples/images/instances for our valuation dataset are 412


100%|████████████████████████████████████████████████████████████████████████████████| 412/412 [12:22<00:00,  1.80s/it]

The achieved accuracy for threshold 0.5 was 0.6914414414414415
The achieved recall for threshold 0.5 was 0.7156177156177156
The achieved precision was 0.953416149068323
F1  score is 0.8175765645805592
fp 15 , tn 0, tp 307 , fn 122
Average IoU achieved was 0.7681629804226972%


In [41]:
import sys
import tools_matrix as tools
import cv2, keras
import numpy as np
import matplotlib.pyplot as plt
import time
from MTCNN import create_Kao_Onet, create_Kao_Rnet, create_Kao_Pnet

Rnet_input = []
Onet_input = []

Pnet = create_Kao_Pnet(r'12net.h5')
Rnet = create_Kao_Rnet(r'24net.h5')
Onet = create_Kao_Onet(r'48net.h5')  # will not work. caffe and TF incompatible
'''

Pnet = keras.models.load_model('Pnet_full_model.h5')
Rnet = keras.models.load_model('Rnet_full_model.h5')
Onet = keras.models.load_model('Onet_full_model.h5')
'''
def detectFace(img, threshold):

    caffe_img = (img.copy() - 127.5) / 127.5 #set in the range of [-1,1]
    origin_h, origin_w, ch = caffe_img.shape
    #scales = tools.calculateScales(img) #generate various scales to create the image pyramid
    #print(scales)
    scales = [1,0.98,0.97, 0.6,0.3,0.2,0.1, 0.9976768786552869, 0.7073529069665984, 0.5015132110393182]
    out = []
    t0 = time.time()
    # del scales[:4]

    for scale in scales:
        hs = int(origin_h * scale)
        ws = int(origin_w * scale)
        #scale_img = pad_if_needed(origin_w,origin_h,ws,hs,img)
        scale_img = cv2.resize(caffe_img, (ws, hs))
        input = scale_img.reshape(1, *scale_img.shape)
        ouput = Pnet.predict(input)  # .transpose(0,2,1,3) should add, but seems after process is wrong then.
        Pnet.save("Pnet_fullmodel.h5")
        out.append(ouput)
    #print(out)
    image_num = len(scales)
    rectangles = []
    for i in range(image_num):
        cls_prob = out[i][0][0][:, :,
                   1]  # i = #scale, first 0 select cls score, second 0 = batchnum, alway=0. 1 one hot repr
        roi = out[i][1][0]  #these are all the coordinates of each of the boxes
        out_h, out_w = cls_prob.shape
        out_side = max(out_h, out_w)
        # print('calculating img scale #:', i)
        cls_prob = np.swapaxes(cls_prob, 0, 1)
        roi = np.swapaxes(roi, 0, 2)
        rectangle = tools.detect_face_12net(cls_prob, roi, out_side, 1 / scales[i], origin_w, origin_h, threshold[0])
        rectangles.extend(rectangle)
    rectangles = tools.NMS(rectangles, 0.7, 'iou')

    t1 = time.time()
    #print ('time for 12 net is: ', t1-t0)

    if len(rectangles) == 0:
        return rectangles

    crop_number = 0
    out = []
    predict_24_batch = []
    for rectangle in rectangles:
        crop_img = caffe_img[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])]
        scale_img = cv2.resize(crop_img, (24, 24))
        predict_24_batch.append(scale_img)
        crop_number += 1

    predict_24_batch = np.array(predict_24_batch)
    np.save("Pnet_input.npy",predict_24_batch)

    out = Rnet.predict(predict_24_batch)
    #print(out)

    cls_prob = out[0]  # first 0 is to select cls, second batch number, always =0
    cls_prob = np.array(cls_prob)  # convert to numpy
    roi_prob = out[1]  # first 0 is to select roi, second batch number, always =0
    roi_prob = np.array(roi_prob)
    rectangles = tools.filter_face_24net(cls_prob, roi_prob, rectangles, origin_w, origin_h, threshold[1])
    t2 = time.time()
    #print ('time for 24 net is: ', t2-t1)


    if len(rectangles) == 0:
        return rectangles


    crop_number = 0
    predict_batch = []
    for rectangle in rectangles:
        # print('calculating net 48 crop_number:', crop_number)
        crop_img = caffe_img[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])]
        scale_img = cv2.resize(crop_img, (48, 48))
        predict_batch.append(scale_img)
        crop_number += 1

    predict_batch = np.array(predict_batch)

    output = Onet.predict(predict_batch)
    cls_prob = output[0]
    roi_prob = output[1]
    pts_prob = output[2]  # index
    # rectangles = tools.filter_face_48net_newdef(cls_prob, roi_prob, pts_prob, rectangles, origin_w, origin_h,
    #                                             threshold[2])
    rectangles = tools.filter_face_48net(cls_prob, roi_prob, pts_prob, rectangles, origin_w, origin_h, threshold[2])
    t3 = time.time()
    #print ('time for 48 net is: ', t3-t2)

    return rectangles

In [2]:
def IoU(px1,py1,px2,py2 ,tx1,ty1,tx2,ty2):
    '''This function will print the IoU(Intersction Over Union ) for the predicted and ground truth boxes
       It will input the predicted and ground truth coordinates and calculate the Overlapping Area of them
       and the Area of Intersection
    Input: predicted x1, predicted y1, predicted x2, predicted y2, true x1, truey1, truex2, truey1 
       
    Return: IoU (number that shows the amount of overlapping between two rectangles)
            IoU = Intersection / Union = min /max
    '''
    #Compute intersection coordinates
    I_boxX1= min(px1,tx1)
    I_boxY1= min(py1,ty1)
    I_boxX2= min(px2,tx2)
    I_boxY2 =min(py2,tx2)
    
    #Compute intersection Area
    interArea = max(0, I_boxX2 - I_boxX1 + 1) * max(0, I_boxY2 - I_boxY1 + 1)
    
    #Compute Union : A U B= A + B - (A ^ B )
    AreaP = max(0, px2-px1+1)*max(0,py2-py1+1)
    AreaT = max(0,tx2-tx1+1)*max(0,ty2-ty1+1)
    AreaAplusB = AreaP+AreaT
    AreaUnion = AreaAplusB - interArea
    
    
    #Compute IoU
    IoU = interArea/AreaUnion
    
    return IoU
    
    


In [3]:
import random
import cv2
def save_all(image, px1,py1,px2,py2, tx1,ty1,tx2,ty2,co):
    px1 = int(px1)
    px2 = int(px2)
    py1 = int(py1)
    py2 = int(py2)
    #print(px1,py1,px2,py2, tx1,tx2,ty1,ty2)
    cv2.rectangle(image,(tx1,ty1),(tx2,ty2),(255,0,0),1)
    cv2.rectangle(image,(px1,py1),(px2,py2),(0,0,255),1)
    #cv2.putText(image,'C:'+str(co), (px1,y1), cv2.FONT_HERSHEY_SIMPLEX,2, (209, 80, 0, 255),3)
    #cv2.imshow("test",image)
    i = random.randint(1,1000)
    cv2.imwrite("test"+str(i)+".jpg",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
import cv2
def pad_if_needed(i_w,i_h,o_w,o_h,img):
    color = (200,50,0)
    result = np.full((i_h,i_w,3),color,dtype=(img.dtype)) #make a blue image
    
    # compute center offset
    #xx = (i_w-o_w) // 2
    #yy = (i_h-o_h)// 2
    
    #new size of img
    img = cv2.resize(img,(o_w,o_h))
    
    
    # copy img image into center of result image
    result[0:o_h, 0:o_w] = img
   
    return result

# Make and Write Image Pyramid with Padding

In [19]:
import cv2
import tools_matrix as tools
import random
import numpy as np
#For Mitsotakis image
img = cv2.imread("D:/Desktop/Microcontroller-Image-Sender-master/test_images/mitsotakis.jpg")
#origin_h,origin_w, ch = img.shape
origin_h ,origin_w = (300,300)
scales = tools.calculateScales(img)
for scale in scales:
    if scale<1.0:
    hs = int(origin_h * scale)
    ws = int(origin_w * scale)
    padded_img = pad_if_needed(origin_w,origin_h,ws,hs,img)
    i = random.randint(1,1000)
    cv2.imwrite('mistotakis'+str(i)+'.jpg', padded_img)
    